In [1]:
#import required libraries

import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import numpy as np 
import warnings
import random
from collections import Counter
# !pip install wordcloud
# from wordcloud import Wordcloud
import nltk
from nltk.stem import PorterStemmer 
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding,Dropout,Bidirectional,LSTM,Dense
from tensorflow.keras.models import Sequential 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import bz2

warnings.filterwarnings("ignore")

c:\Users\swath\Desktop\Amazon Reviews for sentiment analysis\myenv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
train_data=bz2.BZ2File("input//amazonreviews//train.ft.txt.bz2")
test_data=bz2.BZ2File("input//amazonreviews//test.ft.txt.bz2")


In [3]:
train_data=train_data.readlines()
test_data=test_data.readlines()

In [4]:
# train_data.shape

In [5]:
train_data[0]

b'__label__2 Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^\n'

In [6]:
max(train_data)

b'__label__2 \xe2\x99\xa6 LOVE IT \xe2\x99\xa6: Wow I absolutely love this book!!! I definately like this series ALOT better than the Harry Potter series(and i absolutely love Harry Potter)!!!\n'

In [7]:
len(test_data)

400000

In [8]:
train_file_lines = [x.decode('utf-8') for x in train_data]
test_file_lines = [x.decode('utf-8') for x in test_data]

In [9]:
#separate labels and sentences
train_labels=[0 if x.split(" ")[0]=="__label__1" else 1 for x in train_file_lines]

train_sentences=[x.split(" ",1)[1][:-1].lower() for x in train_file_lines]

In [10]:
len(train_sentences)

3600000

In [11]:
import re

In [12]:
for i in range(len(train_sentences)):
    train_sentences[i] = re.sub('\d','0',train_sentences[i])

In [13]:
#separate test labels and senetences
test_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test_file_lines]
test_sentences = [x.split(' ', 1)[1][:-1].lower() for x in test_file_lines]

In [14]:
for i in range(len(test_sentences)):
    test_sentences[i] = re.sub('\d','0',test_sentences[i])

In [15]:
#replacing urls to <url>
for i in range(len(train_sentences)):
    if 'www.' in train_sentences[i] or 'http:' in train_sentences[i] or 'https:' in train_sentences[i] or '.com' in train_sentences[i]:
        train_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", train_sentences[i])
        
for i in range(len(test_sentences)):
    if 'www.' in test_sentences[i] or 'http:' in test_sentences[i] or 'https:' in test_sentences[i] or '.com' in test_sentences[i]:
        test_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", test_sentences[i])

In [16]:
max_features=20000
maxlen=100

In [17]:
train_sentences[0]

'stuning even for the non-gamer: this sound track was beautiful! it paints the senery in your mind so well i would recomend it even to people who hate vid. game music! i have played the game chrono cross but out of all of the games i have ever played it has the best music! it backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. it would impress anyone who cares to listen! ^_^'

In [18]:
tokenizer=Tokenizer(num_words=max_features)

In [19]:
tokenizer.fit_on_texts(train_sentences)

In [20]:
tokenized_train=tokenizer.texts_to_sequences(train_sentences)
x_train=pad_sequences(tokenized_train,maxlen)

In [21]:
tokenized_test=tokenizer.texts_to_sequences(test_sentences)
x_test=pad_sequences(tokenized_test,maxlen)

In [22]:
tokenizer.texts_to_sequences("hello madam")

[[2302], [638], [1873], [1873], [1066], [], [1513], [4], [808], [4], [1513]]

In [23]:
len(train_labels)

3600000

In [24]:
#let's make the data small 

MAX_SAMPLES = 200_000   # even this is large

x_train_small = x_train[:MAX_SAMPLES]
y_train_small = train_labels[:MAX_SAMPLES]

x_test_small = x_test[:50_000]
y_test_small = test_labels[:50_000]

In [25]:
import numpy as np

x_train_small = np.asarray(x_train_small, dtype="int32")
x_test_small  = np.asarray(x_test_small, dtype="int32")

y_train_small = np.asarray(y_train_small, dtype="int32")
y_test_small  = np.asarray(y_test_small, dtype="int32")


In [28]:
y_train_small = y_train_small.reshape(-1,)
y_test_small  = y_test_small.reshape(-1,)


In [29]:
model=Sequential()
model.add(Embedding(20000,200,input_length=100))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(16000, activation='softmax'))
model.add(Dense(1,activation="sigmoid"))
adam = Adam()
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(x_train_small,y_train_small, epochs=2, batch_size=32,verbose=1,validation_data=(x_test_small,y_test_small))
#print model.summary()
print(model)

Epoch 1/2
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 1625s 259ms/step - accuracy: 0.7848 - loss: 0.4292 - val_accuracy: 0.9020 - val_loss: 0.2516
Epoch 2/2
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 3719s 595ms/step - accuracy: 0.9200 - loss: 0.2108 - val_accuracy: 0.9087 - val_loss: 0.2286
<Sequential name=sequential_1, built=True>


In [30]:
model.evaluate(x_test_small,y_test_small)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 107s 68ms/step - accuracy: 0.9087 - loss: 0.2286


[0.22862976789474487, 0.9087399840354919]

In [32]:
pred=model.predict(x_test_small)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 101s 65ms/step


In [35]:
confusion_matrix(y_test_small,pred.argmax(axis=1))

array([[24626,     0],
       [25374,     0]])

In [36]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test_small,pred.argmax(axis=1))

0.49252

x_train shape: (3600000, 100)
x_train dtype: int32
x_test shape: (400000, 100)
x_test dtype: int32


AttributeError: 'list' object has no attribute 'shape'